# BERT End to End (Fine-tuning + Predicting) in 5 minutes with Cloud TPU

## Overview

**BERT**, or **B**idirectional **E**mbedding **R**epresentations from **T**ransformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. The academic paper can be found here: https://arxiv.org/abs/1810.04805.

This Colab demonstates using a free Colab Cloud TPU to fine-tune sentence and sentence-pair classification tasks built on top of pretrained BERT models and 
run predictions on tuned model. The colab demonsrates loading pretrained BERT models from both [TF Hub](https://www.tensorflow.org/hub) and checkpoints.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. You have [$300 free credit](https://cloud.google.com/free/) to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this notebook, you will learn how to train and evaluate a BERT model using TPU.

## Instructions

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

   1. Create a Cloud Storage bucket for your TensorBoard logs at http://console.cloud.google.com/storage and fill in the BUCKET parameter in the "Parameters" section below.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All** (Watch out: the "Colab-only auth for this notebook and the TPU" cell requires user input). You can also run the cells manually with Shift-ENTER.

### Set up your TPU environment

In this section, you perform the following tasks:

*   Set up a Colab TPU running environment
*   Verify that you are connected to a TPU device
*   Upload your credentials to TPU to access your GCS bucket.

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.29.214.2:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15424681047144544699),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13813547154624671566),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12869142696822744466),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 12651457877795909178),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12636266837984255050),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15850439552621819954),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 6773820945566904616),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15789355259147257011),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 9519591032

W0716 20:05:27.122159 139877051504512 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### Prepare and import BERT modules
​
With your environment configured, you can now prepare and import the BERT modules. The following step clones the source code from GitHub and import the modules from the source. Alternatively, you can install BERT using pip (!pip install bert-tensorflow).

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

W0716 20:05:32.011282 139877051504512 deprecation_wrapper.py:119] From bert_repo/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



###Preparação do dataset e conversão para formato BERT

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import re

In [0]:
!unzip df_train.csv.zip
!unzip df_valid.csv.zip

Archive:  df_train.csv.zip
replace df_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: df_train.csv            
Archive:  df_valid.csv.zip
replace df_valid.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: df_valid.csv            


In [0]:
le = LabelEncoder()
 
df = pd.read_csv("df_train.csv")
df.head()

,title,text,category
0,Casa da Barra Funda tem clima roceiro e receit...,"No Quem Quer Pão 75, casa de clima roceiro em ...",comida
1,Professores de SP decidem manter greve; grupo ...,Professores da rede estadual decidiram manter ...,educacao
2,"Em segunda edição, concurso paga R$ 35 mil par...",O Prêmio FedEx para Pequenas Empresas está com...,empreendedorsocial
3,Usar maconha por anos não faz tão mal para a s...,No esforço de entender os efeitos da maconha s...,equilibrioesaude
4,Baleia-azul percorre 5.200 km e revela a cient...,"Isabela, uma baleia-azul, ajudou os cientistas...",ciencia


In [0]:
df['text'] = df['text'].str.strip()
df['title']=df['title'].str.lower()
df['text']=df['text'].str.lower()
df["title"] = df['title'].str.replace('[^\w\s]','')
df["text"] = df['text'].str.replace('[^\w\s]','')

#Concatenando title e text
df['textao'] = df.apply(lambda x: x['title'] + ' ' + x['text'], 1)
df = df.drop(['title', 'text'], 1)

#pegando apenas os 150 primeiros caracteres 
#df["textao"] = df["textao"].str.slice(0,150)

df.head()


,category,textao
0,comida,casa da barra funda tem clima roceiro e receit...
1,educacao,professores de sp decidem manter greve grupo f...
2,empreendedorsocial,em segunda edição concurso paga r 35 mil para ...
3,equilibrioesaude,usar maconha por anos não faz tão mal para a s...
4,ciencia,baleiaazul percorre 5200 km e revela a cientis...


In [0]:
# Creating train and dev dataframes according to BERT
df_bert = pd.DataFrame({'user_id':range(len(df)),
            'label':le.fit_transform(df['category']), #encoder da categoria para número 0,1,2,3...
            'alpha':['a']*df.shape[0],
            'text':df['textao'].replace(r'\n',' ',regex=True).replace(r'\t','',regex=True).replace(r'\"','',regex=True).replace(r'\'','',regex=True)})
 
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.25)
df_bert_train.head()

,user_id,label,alpha,text
1037,1037,5,a,cães da usp escapam de distrofia muscular e po...
191,191,5,a,mães que trabalham fora têm filhos mais educad...
4254,4254,1,a,análise de dna de esqueletos muda teoria sobre...
7122,7122,6,a,novo brooklin chega à vila cordeiro próxima a...
5213,5213,8,a,pontos turísticos da itália são fechados depoi...


In [0]:
# Creating test dataframe according to BERT
df_test = pd.read_csv("df_valid.csv")
df_test['text'] = df_test['text'].str.strip()
df_test['title']=df_test['title'].str.lower()
df_test['text']=df_test['text'].str.lower()
df_test["title"] = df_test['title'].str.replace('[^\w\s]','')
df_test["text"] = df_test['text'].str.replace('[^\w\s]','')

#df_test['textao'] = df_test.apply(lambda x: x['title'] + ' ' + x['text'], 1)
#df_test = df_test.drop(['title', 'text'], 1)

#pegando apenas os 300 primeiros caracteres 
#df_test["textao"] = df["textao"].str.slice(0,300)

df_bert_test = pd.DataFrame({'User_ID':range(len(df_test)),
                 'text':df_test['text'].replace(r'\n',' ',regex=True).replace(r'\t','',regex=True).replace(r'\"','',regex=True).replace(r'\'','',regex=True)})
 
# Saving dataframes to .tsv format as required by BERT
df_bert_train.to_csv('train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('test.tsv', sep='\t', index=False, header=True)

### Prepare for training

This next section of code performs the following tasks:

*  Specify task and download training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.




In [0]:
TASK = 'CoLA' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

# Download glue data.
#! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
#!python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$TASK
#!unzip glue_data.zip

TASK_DATA_DIR = './'
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

BUCKET = 'pln-bert01' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
#   bert_multi_cased_L-12_H-768_A-12: novo
BERT_MODEL = 'multi_cased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Task data directory: ./ *****
adc.json   df_train.csv      df_valid.csv.zip	   test_results.tsv
bert_repo  df_train.csv.zip  sample_data	   test.tsv
dev.tsv    df_valid.csv      submission_valid.csv  train.tsv
***** Model output directory: gs://pln-bert01/bert-tfhub/models/CoLA *****


Now let's load tokenizer module from TF Hub and play with it.

In [0]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("um EXEMPLO de tokenização com a rede Bert")

['um',
 'EX',
 '##EM',
 '##P',
 '##L',
 '##O',
 'de',
 'tok',
 '##eni',
 '##zação',
 'com',
 'a',
 'rede',
 'Bert']

Also we initilize our hyperprams, prepare the training data and initialize TPU config.

In [0]:
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-6
NUM_TRAIN_EPOCHS = 14.0
MAX_SEQ_LENGTH = 200
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.18
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


# Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

This section demonstrates fine-tuning from a pre-trained BERT TF Hub module and running predictions.


In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


W0716 20:46:26.754944 139877051504512 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f375553bf28>) includes params argument, but params are not passed to Estimator.


At this point, you can now fine-tune the model, evaluate it, and run predictions on it.

In [0]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [0]:
model_train(estimator_from_tfhub)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
***** Started training at 2019-07-16 20:47:36.857741 *****
  Num examples = 5920
  Batch size = 16


E0716 20:47:50.881925 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:47:50.884044 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:47:50.887786 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:47:50.890632 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:47:50.901534 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

***** Finished training at 2019-07-16 20:52:23.392451 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [0]:
model_eval(estimator_from_tfhub)

***** Started evaluation at 2019-07-16 20:53:14.877564 *****
  Num examples = 1974
  Batch size = 8


E0716 20:53:24.321354 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:53:24.323379 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:53:24.325229 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:53:24.326941 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0716 20:53:24.334302 139877051504512 tpu.py:376] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

***** Finished evaluation at 2019-07-16 20:54:13.862395 *****
***** Eval results *****
  eval_accuracy = 0.9273374
  eval_loss = 0.41927478
  global_step = 5180
  loss = 0.18559228


In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_test_examples(TASK_DATA_DIR)
  #[:PREDICT_BATCH_SIZE]
  #prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  for example, prediction in zip(prediction_examples, predictions):
    print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))


In [0]:
model_predict(estimator_from_tfhub) 

# Fine-tune and run predictions on a pre-trained BERT model from checkpoints

Alternatively, you can also load pre-trained BERT models from saved checkpoints.

In [0]:
# Setup task specific model and TPU running config.
BERT_PRETRAINED_DIR = 'gs://pln-bert01/bert/' + BERT_MODEL 
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

model_fn = run_classifier.model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

OUTPUT_DIR = OUTPUT_DIR.replace('bert-tfhub', 'bert-checkpoints')
tf.gfile.MakeDirs(OUTPUT_DIR)

estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

***** BERT pretrained directory: gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12 *****
gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12/bert_config.json
gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12/bert_model.ckpt.index
gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12/vocab.txt
gs://pln-bert01/bert/multi_cased_L-12_H-768_A-12/vocab_teste.txt


W0716 20:54:38.843158 139877051504512 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f375553b950>) includes params argument, but params are not passed to Estimator.


Now, you can repeat the training, evaluation, and prediction steps.

In [0]:
model_train(estimator_from_checkpoints)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
***** Started training at 2019-07-16 20:55:47.538873 *****
  Num examples = 5920
  Batch size = 16
***** Finished training at 2019-07-16 21:00:29.951544 *****


In [0]:
model_eval(estimator_from_checkpoints)

***** Started evaluation at 2019-07-16 21:00:56.832073 *****
  Num examples = 1974
  Batch size = 8
***** Finished evaluation at 2019-07-16 21:01:44.610197 *****
***** Eval results *****
  eval_accuracy = 0.9324187
  eval_loss = 0.38113514
  global_step = 5180
  loss = 0.18948396


In [0]:
model_predict(estimator_from_checkpoints)

NameError: ignored

In [0]:
def model_predict_final(estimator):
  #Make predictions on a subset of eval examples
  prediction_examples = processor.get_test_examples(TASK_DATA_DIR)#[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  
  return(predictions)

In [0]:
result = model_predict_final(estimator_from_checkpoints)
output_predict_file = ('test_results.tsv')
with tf.gfile.GFile(output_predict_file, "w") as writer:
  num_written_lines = 0
  tf.logging.info("***** Predict results *****")
  for prediction in result:
    probabilities = prediction["probabilities"]
    output_line = "\t".join(
        str(class_probability)
        for class_probability in probabilities) + "\n"
    writer.write(output_line)
    num_written_lines += 1
print(num_written_lines)

4251


In [0]:
predictions = pd.read_csv('test_results.tsv', header=None, sep='\t')

df_results_csv = pd.DataFrame({'id':range(len(predictions)), 'category':le.inverse_transform(predictions.idxmax(axis=1))})
df_result_matriz_conf = pd.DataFrame({'category':predictions.idxmax(axis=1)})

# writing into .csv
df_results_csv.to_csv('submission_valid.csv',index=None)
df_results_csv.head()
print(len(df_results_csv))

4251
